In [27]:
# Imports
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold
from scikeras.wrappers import KerasRegressor
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE
import pandas as pd
import numpy as np
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import warnings
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error
import tensorflow as tf
import torch
import torch.nn as nn
from pytorch_forecasting.models import TemporalFusionTransformer
from torchmetrics import MeanSquaredError
from pytorch_lightning import LightningModule
import torch
import torch.optim as optim
from torchmetrics import MeanSquaredError
from pytorch_lightning import LightningModule
from pytorch_forecasting.models import TemporalFusionTransformer
from pytorch_lightning import Trainer
from torch.utils.data import DataLoader
import lightning.pytorch as pl # Instead of import pytorch_lightning as pl

from lightning.pytorch.callbacks.early_stopping import EarlyStopping
from lightning.pytorch.callbacks import LearningRateMonitor
from pytorch_forecasting import DeepAR, TimeSeriesDataSet
from pytorch_forecasting.data import GroupNormalizer
from pytorch_lightning import Trainer
from torch.utils.data import DataLoader

# Assuming 'train_data' and 'test_data' are already defined and preprocessed as per your previous code

#




In [28]:
# Load SP500 and US macroeconomic data from CSV files
data = pd.read_csv('./data/SP500_data/GOOGL.csv', parse_dates=['Date'])
macro_data = pd.read_csv('./data/SP500_data/US_macroeconomics.csv', parse_dates=['date'])

data['Date'] = pd.to_datetime(data['Date'])
data.set_index('Date', inplace=True)  # Set the date column as the index if not already set

# Merge datasets on the date field
merged_data = pd.merge(data, macro_data, left_on='Date', right_on='date', how='outer')

# Handle missing values: forward fill then backward fill to cover all gaps
merged_data.interpolate(method='ffill', inplace=True)
merged_data.interpolate(method='bfill', inplace=True)

# Drop the extra date column to avoid duplication
merged_data.drop(columns=['date'], inplace=True)

print(merged_data)



             Open         High          Low        Close    Adj Close  \
0       50.050049    52.082081    48.028027    50.220219    50.220219   
1       50.050049    52.082081    48.028027    50.220219    50.220219   
2       50.050049    52.082081    48.028027    50.220219    50.220219   
3       50.050049    52.082081    48.028027    50.220219    50.220219   
4       50.050049    52.082081    48.028027    50.220219    50.220219   
...           ...          ...          ...          ...          ...   
4787  2722.030029  2821.000000  2722.030029  2797.360107  2797.360107   
4788  2774.050049  2791.770020  2756.699951  2765.510010  2765.510010   
4789  2784.000000  2832.379883  2755.010010  2831.439941  2831.439941   
4790  2784.000000  2832.379883  2755.010010  2831.439941  2831.439941   
4791  2784.000000  2832.379883  2755.010010  2831.439941  2831.439941   

          Volume      CPI  Mortgage_rate  Unemp_rate        NASDAQ  \
0     44659096.0   85.600        14.2050         7.5 

/var/folders/d9/x_bnq71n58lbk4_znq_twz6h0000gn/T/ipykernel_11364/1542437413.py:12: FutureWarning: DataFrame.interpolate with method=ffill is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  merged_data.interpolate(method='ffill', inplace=True)
/var/folders/d9/x_bnq71n58lbk4_znq_twz6h0000gn/T/ipykernel_11364/1542437413.py:13: FutureWarning: DataFrame.interpolate with method=bfill is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  merged_data.interpolate(method='bfill', inplace=True)


In [29]:
# Import necessary modules
from pytorch_forecasting.data import TimeSeriesDataSet

# Interpolating missing data using forward and backward filling
merged_data = merged_data.ffill().bfill()

# Convert all float data to float32 to ensure compatibility with MPS
for col in merged_data.select_dtypes(include=['float64']):
    merged_data[col] = merged_data[col].astype('float32')


# Limit the dataset to the first 1000 rows
limited_data = merged_data.head(100)

# Split the limited data into train and test sets
train_data = limited_data[:int(0.8 * len(limited_data))]
test_data = limited_data[int(0.8 * len(limited_data)):]

# Display the shapes of the train and test data
print("Train data shape:", train_data.shape)
print("Test data shape:", test_data.shape)
train_data = train_data.copy()

# Add time_idx and group_id columns for the training data
train_data.loc[:, 'time_idx'] = range(len(train_data))
train_data.loc[:, 'group_id'] = 1

print(train_data.to_string())
print("Checking 'time_idx' column in DataFrame:")
print("Column exists:", "time_idx" in train_data.columns)
print("Data type of 'time_idx':", train_data['time_idx'].dtype)

# Define the TimeSeriesDataSet for training
# Define the TimeSeriesDataSet for training
train_dataset = TimeSeriesDataSet(
    data=train_data,
    time_idx="time_idx",
    group_ids=["group_id"],
    target="Close",
    max_encoder_length=3,  # Adjust to suit your needs
    max_prediction_length=5,  # Adjust to suit your needs

    time_varying_known_reals=["Open", "High", "Low", "Volume"],  # Example time-varying known reals
    time_varying_unknown_reals=["Close"],  # Target variable in the unknown reals
    allow_missing_timesteps=False,
)


print("Data type of 'Close':", train_data['Close'].dtype)



Train data shape: (80, 13)
Test data shape: (20, 13)
         Open       High        Low      Close  Adj Close      Volume         CPI  Mortgage_rate  Unemp_rate      NASDAQ  disposable_income  Personal_consumption_expenditure  personal_savings  time_idx  group_id
0   50.050049  52.082081  48.028027  50.220219  50.220219  44659096.0   85.599998      14.205000         7.5  200.685562        4976.500000                       1826.800049              11.6         0         1
1   50.050049  52.082081  48.028027  50.220219  50.220219  44659096.0   86.400002      14.790000         7.2  198.398636        4999.799805                       1851.699951              11.4         1         1
2   50.050049  52.082081  48.028027  50.220219  50.220219  44659096.0   87.199997      14.904000         7.5  198.817612        4980.399902                       1870.000000              10.9         2         1
3   50.050049  52.082081  48.028027  50.220219  50.220219  44659096.0   88.000000      15.132500   

In [33]:
# Define the DeepAR model using previous timeseries data
deepar = DeepAR.from_dataset(
    train_dataset,  # Using the previously defined TimeSeriesDataSet
    learning_rate=0.001,
    hidden_size=64,
    rnn_layers=2,
    dropout=0.1,
    loss="NormalDistributionLoss",
    output_size=1
)

# Define PyTorch Lightning trainer
trainer = Trainer(
    max_epochs=50,
    gpus=1,  # Set according to your system's configuration
    gradient_clip_val=0.1
)

# Fit the model using the previously defined filtered dataset
trainer.fit(deepar, train_dataloader=DataLoader(train_dataset, batch_size=32),)




/Users/almazkhalilov/Library/Python/3.9/lib/python/site-packages/lightning/pytorch/utilities/parsing.py:199: Attribute 'logging_metrics' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['logging_metrics'])`.


AssertionError: number of targets should be equivalent to number of loss metrics

In [ ]:
from pytorch_forecasting.models import TemporalFusionTransformer
from pytorch_lightning import Trainer, LightningModule
from torch.utils.data import DataLoader

torch.set_default_dtype(torch.float32)

class TFTLightningModule(LightningModule):
    def __init__(self, model):
        super().__init__()
        self.model = model
    
    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        # Ensure batch is a tuple (x, y) or dictionary with appropriate keys
        x = batch["x"] if isinstance(batch, dict) else batch[0]
        y = batch["y"] if isinstance(batch, dict) else batch[1]
        
        # Generate predictions and calculate loss
        y_hat = self(x)
        loss = self.model.loss(y_hat, y)
        return loss

    def configure_optimizers(self):
        # You can customize this optimizer or use the model's defaults
        optimizer = optim.Adam(self.parameters(), lr=0.01)
        return optimizer

# Define the TFT model from the training dataset
tft_model = TemporalFusionTransformer.from_dataset(
    dataset=train_dataset,
    learning_rate=0.01,  # You can adjust the learning rate as needed
    hidden_size=16,  # Hidden layer size
    attention_head_size=4,  # Number of attention heads
    dropout=0.1,  # Dropout rate to prevent overfitting
    hidden_continuous_size=8,  # Size of continuous variable hidden layer
    output_size=1,  # Number of outputs (e.g., 1 for regression tasks)
)

# Wrap the TFT model inside a LightningModule
tft_lightning_module = TFTLightningModule(tft_model)

# Setup a PyTorch Lightning Trainer
trainer = Trainer(
    max_epochs=30,  # Specify the number of epochs for training
    gradient_clip_val=0.1,  # Gradient clipping to prevent exploding gradients
)

# Train the model
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)  # Adjust batch_size and shuffle as needed
trainer.fit(tft_lightning_module, train_dataloaders=train_loader)

/Users/almazkhalilov/Library/Python/3.9/lib/python/site-packages/lightning/pytorch/utilities/parsing.py:199: Attribute 'loss' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss'])`.
/Users/almazkhalilov/Library/Python/3.9/lib/python/site-packages/lightning/pytorch/utilities/parsing.py:199: Attribute 'logging_metrics' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['logging_metrics'])`.
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name  | Type                      | Params
----------------------------------------------------
0 | model | TemporalFusionTransformer | 19.6 K
----------------------------------------------------
19.6 K    Trainable params
0         Non-trainable params


Epoch 0:   0%|          | 0/3 [00:00<?, ?it/s] 

TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found <class 'NoneType'>

In [ ]:
# Iterate through the data loader
for i, batch in enumerate(train_loader):
    print(f"Batch {i}:")
    print("  Encoder categorical:", batch["encoder_cat"])
    print("  Encoder continuous:", batch["encoder_cont"])
    print("  Target:", batch["target"])

    # Stop after a few batches
    if i >= 2: 
        break


for i in range(3): # Change 3 to check more rows
    print(f"Row {i}:")
    print(train_dataset[i])


Batch 0:
  Encoder categorical: tensor([[[1.7270e+02, 8.0275e+00, 4.1000e+00, 3.9096e+03, 9.5915e+03,
          6.8028e+03, 5.2000e+00]],

        [[1.9930e+02, 6.1450e+00, 4.7000e+00, 2.2900e+03, 1.1097e+04,
          9.0716e+03, 3.7000e+00]],

        [[1.0910e+02, 1.0080e+01, 7.2000e+00, 3.6823e+02, 6.0536e+03,
          2.8236e+03, 9.9000e+00]],

        [[1.9910e+02, 6.0650e+00, 5.0000e+00, 2.0871e+03, 1.0870e+04,
          8.9385e+03, 2.9000e+00]],

        [[2.0180e+02, 6.6820e+00, 4.6000e+00, 2.1374e+03, 1.1198e+04,
          9.2596e+03, 3.7000e+00]],

        [[1.7460e+02, 7.3820e+00, 3.9000e+00, 2.6578e+03, 9.6476e+03,
          6.9522e+03, 4.4000e+00]],

        [[1.5090e+02, 8.8275e+00, 5.4000e+00, 7.8424e+02, 7.6691e+03,
          4.8508e+03, 7.8000e+00]],

        [[2.0290e+02, 6.7625e+00, 4.7000e+00, 2.0862e+03, 1.1196e+04,
          9.3438e+03, 3.1000e+00]],

        [[1.0750e+02, 1.2215e+01, 7.4000e+00, 2.9044e+02, 5.8448e+03,
          2.6894e+03, 9.6000e+00]],

     

In [ ]:




# Define the TFT model
class StockPriceTFT(LightningModule):
    def __init__(self, config):
        super().__init__()
        self.config = config  # Directly assign the config
        self.save_hyperparameters(ignore=["logging_metrics"])  # Note the exclusion
        self.tft = TemporalFusionTransformer(
            hidden_size=config["hidden_size"],
            lstm_layers=config["lstm_layers"],
            dropout=config["dropout"],
            output_size=config["output_size"],
            attention_head_size=config["attention_head_size"],
            max_encoder_length=config["max_encoder_length"],
            static_categoricals=config["static_categoricals"],
            static_reals=config["static_reals"],
            time_varying_categoricals_encoder=config["time_varying_categoricals_encoder"],
            time_varying_categoricals_decoder=config["time_varying_categoricals_decoder"],
            categorical_groups=config["categorical_groups"],
            time_varying_reals_encoder=config["time_varying_reals_encoder"],
            time_varying_reals_decoder=config["time_varying_reals_decoder"],
            x_reals=config["x_reals"],
            x_categoricals=config["x_categoricals"],
            hidden_continuous_size=config["hidden_continuous_size"],
            hidden_continuous_sizes=config["hidden_continuous_sizes"],
            embedding_sizes=config["embedding_sizes"],
            embedding_paddings=config["embedding_paddings"],
            embedding_labels=config["embedding_labels"],
            learning_rate=config["learning_rate"],
            log_interval=config["log_interval"],
            log_val_interval=config["log_val_interval"],
            log_gradient_flow=config["log_gradient_flow"],
            reduce_on_plateau_patience=config["reduce_on_plateau_patience"],
            monotone_constaints=config["monotone_constaints"],
            share_single_variable_networks=config["share_single_variable_networks"],
            causal_attention=config["causal_attention"],
            logging_metrics=config["logging_metrics"]
        )
        self.loss_fn = MeanSquaredError()


    

    def forward(self, x):
        return self.tft(x)

    def training_step(self, batch, batch_idx):
        output = self(batch)
        loss = self.loss_fn(output, batch["target"])
        return loss

    def validation_step(self, batch, batch_idx):
        output = self(batch)
        loss = self.loss_fn(output, batch["target"])
        self.log("val_loss", loss)
        return loss

    def configure_optimizers(self):
        return optim.Adam(self.parameters(), lr=self.hparams.config["learning_rate"])

def custom_serialize(obj):
    # Custom serialization logic
    return str(obj)
# Configure the TFT model
config = {
    "hidden_size": 64,
    "lstm_layers": 2,
    "dropout": 0.1,
    "output_size": 1,
    "attention_head_size": 4,
    "max_encoder_length": 30,
    "static_categoricals": [],
    "static_reals": [],
    "time_varying_categoricals_encoder": [],
    "time_varying_categoricals_decoder": [],
    "categorical_groups": {},
    "time_varying_reals_encoder": ["price"],
    "time_varying_reals_decoder": ["price"],
    "x_reals": ["price"],
    "x_categoricals": [],
    "hidden_continuous_size": 16,
    "hidden_continuous_sizes": {},
    "embedding_sizes": {},
    "embedding_paddings": [],
    "embedding_labels": {},
    "learning_rate": 0.001,
    "log_interval": 10,
    "log_val_interval": 10,
    "log_gradient_flow": False,
    "reduce_on_plateau_patience": 10,
    "monotone_constaints": {},
    "share_single_variable_networks": False,
    "causal_attention": True,
    "logging_metrics": None
}



# Create the TFT model
model = StockPriceTFT(config)
# Train the model

trainer = Trainer(max_epochs=100)

trainer.fit(model, train_loader, train_loader)


GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name    | Type                      | Params
------------------------------------------------------
0 | tft     | TemporalFusionTransformer | 279 K 
1 | loss_fn | MeanSquaredError          | 0     
------------------------------------------------------
279 K     Trainable params
0         Non-trainable params
279 K     Total params
1.119     Total estimated model params size (MB)


Sanity Checking DataLoader 0:   0%|          | 0/1 [00:00<?, ?it/s]

/Users/almazkhalilov/Library/Python/3.9/lib/python/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:492: Your `val_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.
/Users/almazkhalilov/Library/Python/3.9/lib/python/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


KeyError: 'encoder_lengths'